## 월별 채널별 매출과 방문자 정보 계산하기

In [0]:
df_user_session_channel = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("s3a://s3-geospatial/readonly/user_session_channel.csv")

df_session_timestamp = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("s3a://s3-geospatial/readonly/session_timestamp.csv")

df_session_transaction = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("s3a://s3-geospatial/readonly/session_transaction.csv")

In [0]:
df_user_session_channel.createOrReplaceTempView("user_session_channel")
df_session_timestamp.createOrReplaceTempView("session_timestamp")
df_session_transaction.createOrReplaceTempView("session_transaction")

In [0]:
%sql
SELECT ts, channel
FROM user_session_channel usc
JOIN session_timestamp st ON usc.sessionid = st.sessionid
WHERE userid = 251
ORDER BY 1;

In [0]:
spark.table("session_timestamp").show(5)

+--------------------+--------------------+
|           sessionid|                  ts|
+--------------------+--------------------+
|7cdace91c487558e2...|2019-05-01 00:13:...|
|94f192dee566b018e...|2019-05-01 00:49:...|
|7ed2d3454c5eea711...|2019-05-01 10:18:...|
|f1daf122cde863010...|2019-05-01 13:10:...|
|fd0efcca272f704a7...|2019-05-01 13:45:...|
+--------------------+--------------------+
only showing top 5 rows



In [0]:
df_user_session_channel.show(5)

+------+--------------------+--------+
|userid|           sessionid| channel|
+------+--------------------+--------+
|   184|c41dd99a69df04044...|   Naver|
|    80|fdc0eb412a84fa549...| Organic|
|   251|0a54b19a13b6712dc...|Facebook|
|   264|a914ecef9c12ffdb9...|  Google|
|   744|05ae14d7ae387b933...|Facebook|
+------+--------------------+--------+
only showing top 5 rows



In [0]:
df_session_timestamp.show(5)

+--------------------+--------------------+
|           sessionid|                  ts|
+--------------------+--------------------+
|7cdace91c487558e2...|2019-05-01 00:13:...|
|94f192dee566b018e...|2019-05-01 00:49:...|
|7ed2d3454c5eea711...|2019-05-01 10:18:...|
|f1daf122cde863010...|2019-05-01 13:10:...|
|fd0efcca272f704a7...|2019-05-01 13:45:...|
+--------------------+--------------------+
only showing top 5 rows



In [0]:
df_session_transaction.show(5)

+--------------------+--------+------+
|           sessionid|refunded|amount|
+--------------------+--------+------+
|00029153d12ae1c9a...|   false|    85|
|008909bd27b680698...|   false|    13|
|0107acb41ef20db22...|   false|    16|
|018544a2c48077d2c...|   false|    39|
|020c38173caff0203...|   false|    61|
+--------------------+--------+------+
only showing top 5 rows



## 월별 채널별 총 방문자와 매출 발생 방문자 계산

In [0]:
# JOIN key가 정말 하나씩만 존재하나?
spark.sql("""SELECT sessionid, COUNT(1) count
FROM user_session_channel
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1""").show() 

+--------------------+-----+
|           sessionid|count|
+--------------------+-----+
|5c3a3b139a11689e0...|    1|
+--------------------+-----+



In [0]:
# JOIN key가 정말 하나씩만 존재하나?
spark.sql("""SELECT sessionid, COUNT(1) count
FROM session_transaction
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1""").show() 

+--------------------+-----+
|           sessionid|count|
+--------------------+-----+
|532ff98823e7d1433...|    1|
+--------------------+-----+



In [0]:
# JOIN key가 정말 하나씩만 존재하나?
spark.sql("""SELECT sessionid, COUNT(1) count
FROM session_timestamp
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1""").show() 

+--------------------+-----+
|           sessionid|count|
+--------------------+-----+
|006c64491cb8acf20...|    1|
+--------------------+-----+



### 월별 채널별 총 방문자 계산

In [0]:
spark.sql("""
    SELECT sessionId 
    FROM user_session_channel
    
    EXCEPT
    
    SELECT sessionId
    FROM session_timestamp
""").show()

+---------+
|sessionId|
+---------+
+---------+



In [0]:
spark.sql("""
    SELECT sessionId 
    FROM session_timestamp
    
    EXCEPT
    
    SELECT sessionId
    FROM user_session_channel
""").show()

+---------+
|sessionId|
+---------+
+---------+



In [0]:
mon_channel_rev_df = spark.sql("""
    SELECT LEFT(sti.ts, 7) year_month,
      usc.channel channel,
      COUNT(DISTINCT userid) total_visitors
    FROM user_session_channel usc
    LEFT JOIN session_timestamp sti ON usc.sessionid = sti.sessionid
    GROUP BY 1 ,2
    ORDER BY 1, 2""")

In [0]:
mon_channel_rev_df.show()

### 월별 채널별 총 방문자와 구매 방문자 계산

In [0]:
mon_channel_rev_df = spark.sql("""
    SELECT LEFT(sti.ts, 7) year_month,
      usc.channel channel,
      COUNT(DISTINCT userid) total_visitors,
      COUNT(DISTINCT CASE WHEN amount is not NULL THEN userid END) paid_visitors
    FROM user_session_channel usc
    LEFT JOIN session_timestamp sti ON usc.sessionid = sti.sessionid
    LEFT JOIN session_transaction str ON usc.sessionid = str.sessionid
    GROUP BY 1 ,2
    ORDER BY 1, 2""")

In [0]:
mon_channel_rev_df.show()

+----------+---------+--------------+-------------+
|year_month|  channel|total_visitors|paid_visitors|
+----------+---------+--------------+-------------+
|   2019-05| Facebook|           247|           14|
|   2019-05|   Google|           253|           10|
|   2019-05|Instagram|           234|           11|
|   2019-05|    Naver|           237|           11|
|   2019-05|  Organic|           238|           17|
|   2019-05|  Youtube|           244|           10|
|   2019-06| Facebook|           414|           22|
|   2019-06|   Google|           412|           13|
|   2019-06|Instagram|           410|           21|
|   2019-06|    Naver|           398|           15|
|   2019-06|  Organic|           416|           14|
|   2019-06|  Youtube|           400|           17|
|   2019-07| Facebook|           558|           32|
|   2019-07|   Google|           556|           21|
|   2019-07|Instagram|           567|           25|
|   2019-07|    Naver|           553|           19|
|   2019-07|

### 월별 채널별 총 매출액 (리펀드 포함), 순매출액 (리펀드 제외), 구매자 비율 계산

In [0]:
# 먼저 총 매출액부터 계산
spark.sql("""
  SELECT LEFT(sti.ts, 7) year_month,
      usc.channel channel,
      COUNT(DISTINCT userid) total_visitors,
      COUNT(DISTINCT CASE WHEN amount is not NULL THEN userid END) paid_visitors,
      SUM(amount) gross_revenue
   FROM user_session_channel usc
   LEFT JOIN session_timestamp sti ON sti.sessionid = usc.sessionid
   LEFT JOIN session_transaction str ON str.sessionid = usc.sessionid
   GROUP BY 1, 2
   ORDER BY 1, 2;
""").show()

+----------+---------+--------------+-------------+-------------+
|year_month|  channel|total_visitors|paid_visitors|gross_revenue|
+----------+---------+--------------+-------------+-------------+
|   2019-05| Facebook|           247|           14|         1199|
|   2019-05|   Google|           253|           10|          580|
|   2019-05|Instagram|           234|           11|          959|
|   2019-05|    Naver|           237|           11|          867|
|   2019-05|  Organic|           238|           17|         1846|
|   2019-05|  Youtube|           244|           10|          529|
|   2019-06| Facebook|           414|           22|         1578|
|   2019-06|   Google|           412|           13|          947|
|   2019-06|Instagram|           410|           21|         1462|
|   2019-06|    Naver|           398|           15|         1090|
|   2019-06|  Organic|           416|           14|         1129|
|   2019-06|  Youtube|           400|           17|         1042|
|   2019-0

In [0]:
# 다음으로 순매출액 계산
spark.sql("""
  SELECT LEFT(sti.ts, 7) year_month,
      usc.channel channel,
      COUNT(DISTINCT userid) total_visitors,
      COUNT(DISTINCT CASE WHEN amount is not NULL THEN userid END) paid_visitors,
      SUM(amount) gross_revenue,
      SUM(CASE WHEN refunded is not True THEN amount END) net_revenue
   FROM user_session_channel usc
   LEFT JOIN session_timestamp sti ON sti.sessionid = usc.sessionid
   LEFT JOIN session_transaction str ON str.sessionid = usc.sessionid
   GROUP BY 1, 2
   ORDER BY 1, 2;
""").show()

+----------+---------+--------------+-------------+-------------+-----------+
|year_month|  channel|total_visitors|paid_visitors|gross_revenue|net_revenue|
+----------+---------+--------------+-------------+-------------+-----------+
|   2019-05| Facebook|           247|           14|         1199|        997|
|   2019-05|   Google|           253|           10|          580|        580|
|   2019-05|Instagram|           234|           11|          959|        770|
|   2019-05|    Naver|           237|           11|          867|        844|
|   2019-05|  Organic|           238|           17|         1846|       1571|
|   2019-05|  Youtube|           244|           10|          529|        529|
|   2019-06| Facebook|           414|           22|         1578|       1578|
|   2019-06|   Google|           412|           13|          947|        947|
|   2019-06|Instagram|           410|           21|         1462|       1418|
|   2019-06|    Naver|           398|           15|         1090

In [0]:
 # 유료 사용자 비율 계산
spark.sql("""
  SELECT LEFT(sti.ts, 7) year_month,
      usc.channel channel,
      COUNT(DISTINCT userid) total_visitors,
      COUNT(DISTINCT CASE WHEN amount is not NULL THEN userid END) paid_visitors,
      SUM(amount) gross_revenue,
      SUM(CASE WHEN refunded is not True THEN amount END) net_revenue,
      paid_visitors*100/total_visitors conversion_rate
   FROM user_session_channel usc
   LEFT JOIN session_timestamp sti ON sti.sessionid = usc.sessionid
   LEFT JOIN session_transaction str ON str.sessionid = usc.sessionid
   GROUP BY 1, 2
   ORDER BY 1, 2;
""").show()

+----------+---------+--------------+-------------+-------------+-----------+------------------+
|year_month|  channel|total_visitors|paid_visitors|gross_revenue|net_revenue|   conversion_rate|
+----------+---------+--------------+-------------+-------------+-----------+------------------+
|   2019-05| Facebook|           247|           14|         1199|        997| 5.668016194331984|
|   2019-05|   Google|           253|           10|          580|        580| 3.952569169960474|
|   2019-05|Instagram|           234|           11|          959|        770| 4.700854700854701|
|   2019-05|    Naver|           237|           11|          867|        844| 4.641350210970464|
|   2019-05|  Organic|           238|           17|         1846|       1571| 7.142857142857143|
|   2019-05|  Youtube|           244|           10|          529|        529| 4.098360655737705|
|   2019-06| Facebook|           414|           22|         1578|       1578| 5.314009661835748|
|   2019-06|   Google|        

In [0]:
# 유료 사용자 비율을 소수 둘째 짜리까지 퍼센트 계산 
spark.sql("""
  SELECT LEFT(sti.ts, 7) year_month,
      usc.channel channel,
      COUNT(DISTINCT userid) total_visitors,
      COUNT(DISTINCT CASE WHEN amount is not NULL THEN userid END) paid_visitors,
      SUM(amount) gross_revenue,
      SUM(CASE WHEN refunded is not True THEN amount END) net_revenue,
      ROUND(paid_visitors*100/total_visitors, 2) conversion_rate
   FROM user_session_channel usc
   LEFT JOIN session_timestamp sti ON sti.sessionid = usc.sessionid
   LEFT JOIN session_transaction str ON str.sessionid = usc.sessionid
   GROUP BY 1, 2
   ORDER BY 1, 2;
""").show()

+----------+---------+--------------+-------------+-------------+-----------+---------------+
|year_month|  channel|total_visitors|paid_visitors|gross_revenue|net_revenue|conversion_rate|
+----------+---------+--------------+-------------+-------------+-----------+---------------+
|   2019-05| Facebook|           247|           14|         1199|        997|           5.67|
|   2019-05|   Google|           253|           10|          580|        580|           3.95|
|   2019-05|Instagram|           234|           11|          959|        770|            4.7|
|   2019-05|    Naver|           237|           11|          867|        844|           4.64|
|   2019-05|  Organic|           238|           17|         1846|       1571|           7.14|
|   2019-05|  Youtube|           244|           10|          529|        529|            4.1|
|   2019-06| Facebook|           414|           22|         1578|       1578|           5.31|
|   2019-06|   Google|           412|           13|         